In [33]:
import torch
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import numpy as np

In [34]:
transform = transforms.Compose([#transforms.RandomRotation(30),  # data augmentations are great
                                       #transforms.RandomResizedCrop(224),  # but not in this case of map tiles
                                       #transforms.RandomHorizontalFlip(),
                                       transforms.Resize(1000),
                                       #transforms.ToTensor(),
                                       #transforms.Normalize([0.485, 0.456, 0.406], # PyTorch recommends these but in this
                                       #                     [0.229, 0.224, 0.225]) # case I didn't get good results
                                       ])
batch_size = 4
def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    return sample


dataset = datasets.DatasetFolder(
    root='./seti-breakthrough-listen/torchvision_data/',
    loader=npy_loader,
    extensions=('.npy'),
    transform=transform
)

In [35]:



valid_size = .2
num_train = len(dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
np.random.shuffle(indices)

from torch.utils.data.sampler import SubsetRandomSampler
train_idx, test_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
trainloader = torch.utils.data.DataLoader(dataset, sampler=train_sampler, batch_size=1)
testloader = torch.utils.data.DataLoader(dataset, sampler=test_sampler, batch_size=1)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)
#
classes = ('alien', 'no_alien')
import matplotlib.pyplot as plt
import numpy as npy_loader
print(testloader)

In [3]:


# functions to show an image

#
# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()
#
#
# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()
#
# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [37]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(1000, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [36]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#model.cuda()
# optimizer.to('gpu')

In [38]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
#print(len(labels[0]))
print(labels[0])
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(1)))

tensor(1)
GroundTruth:  no_alien


In [39]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cuda"
print(device)
net.to(device)
net.cuda()
torch.cuda.empty_cache()
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        data[0] = np.take(data[0][0], [0,2,4], 0)
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.float()
        inputs = inputs.unsqueeze(0)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

cuda


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
print(loss)

In [ ]:
dataiter = iter(testloader)
data = dataiter.next()
images, labels = data

#images, labels = data[0].to(device), data[1].to(device)
# print images
#imshow(torchvision.utils.make_grid(images))
#print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
data[0] = np.take(data[0][0], [0,2,4], 0)
images, labels = data[0].to(device), data[1].to(device)
images = images.float()
images = images.unsqueeze(0)
print(images)
outputs = net(images)
print(outputs)

In [ ]:
_, predicted = torch.max(outputs, 1)
print(predicted)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:

print(predicted[0])
print(labels[0])


self.fc1 = nn.Linear(63440, 120)

self.fc2 = nn.Linear(120, 84)

self.fc3 = nn.Linear(84, 2)

lr=0.001, momentum=0.9

Accuracy of the network on the 2000 test images: 48 %